In [1]:
%reload_ext jumper_extension

NVIDIA drivers not available. GPU monitoring disabled.
[JUmPER]: Perfmonitor extension loaded.


In [2]:
%matplotlib widget

In [3]:
%perfmonitor_start 1

[JUmPER]: Performance monitoring started (PID: 989973, Interval: 1.0s)


In [4]:
%perfmonitor_enable_perfreports

[JUmPER]: Performance reports enabled for each cell (level: process)


In [5]:
%perfmonitor_resources

[JUmPER]:
  CPUs: 8
    CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7]
  Memory: 23.13 GB
  GPUs: 0


In [6]:
from random import random
import time
import multiprocessing
from multiprocessing import Pool
import sys

import numpy as np
import gc

import os
import time
import threading

import math

def throw_dart(iterations: int) -> int:
    hits = 0
    for i in range(iterations):
        x = random()
        y = random()
        if (x * x) + (y * y) <= 1:
            hits = hits + 1
    return hits


def compute_pi(iterations, process_count):
    pool = Pool(processes=process_count)
    trials_per_process = [int(iterations / process_count)] * process_count

    hits = pool.map(throw_dart, trials_per_process)
    pi = (sum(hits) * 4) / iterations

    print(f"Pi approximation: {pi}")

[JUmPER]: No non-short cells found


In [7]:
# Cell 1: Utilizing all CPUs for 10**8 iterations
compute_pi(10**8, multiprocessing.cpu_count())

Pi approximation: 3.14149872
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 5.92s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  62.64    0.00     98.31    -       
Memory (GB)               0.04     0.02     0.05     23.13   


In [8]:
%perfmonitor_plot

In [9]:
# Cell 2: Utilizing half of the CPUs for 10**8 iterations
compute_pi(10**8, multiprocessing.cpu_count()//2)

Pi approximation: 3.14158288
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 8.01s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  47.60    37.46    50.68    -       
Memory (GB)               0.05     0.05     0.05     23.13   


In [10]:
%perfmonitor_plot

In [11]:
# Cell 3: Filling 2.5Gb of main memory
def fill_memory_2_5gb():
    """Fill 2.5GB of main memory"""
    print("Filling 2.5GB of main memory...")
    
    # Calculate number of elements for 2.5GB (assuming float64 = 8 bytes each)
    gb_size = 2.5 * 1024 * 1024 * 1024  # 2.5 GB in bytes
    num_elements = int(gb_size / 8)  # 8 bytes per float64 element
    
    print(f"Allocating {num_elements} float64 elements (~{2.5:.2f} GB)")
    
    # Create large arrays
    arrays = []
    try:
        # Create multiple arrays to avoid memory fragmentation issues
        for i in range(10):
            arr = np.ones(num_elements // 10, dtype=np.float64)
            arrays.append(arr)
            print(f"Created array {i+1}")
        
        # Force garbage collection to ensure memory is allocated
        gc.collect()
        
        print("2.5GB memory allocation completed")
        return arrays
        
    except MemoryError:
        print("Memory allocation failed!")
        return []

# Fill memory
allocated_arrays = fill_memory_2_5gb()

Filling 2.5GB of main memory...
Allocating 335544320 float64 elements (~2.50 GB)
Created array 1
Created array 2
Created array 3
Created array 4
Created array 5
Created array 6
Created array 7
Created array 8
Created array 9
Created array 10
2.5GB memory allocation completed
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 2.57s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  12.49    12.49    12.49    -       
Memory (GB)               2.31     2.07     2.55     23.13   


In [12]:
%perfmonitor_plot

In [13]:
# Write 1GB to disk in 10 seconds, force flush, wait 10sec, read back in 10 seconds
def write_and_read_1gb_forced():
    filename = "large_data_forced.bin"
    data_size = 1024 * 1024 * 1024  # 1GB in bytes
    chunk_size = 1024 * 1024  # 1MB chunks
    
    print("Starting 1GB write operation...")
    
    # Calculate write rate: 1GB in 10 seconds = 100MB/s
    target_rate_mb_per_sec = 100  # 100 MB/s
    target_time_per_chunk = 1.0 / target_rate_mb_per_sec  # 0.01 seconds per chunk
    
    start_time = time.time()
    written_bytes = 0
    chunk_count = 0
    
    # Write data with forced flushing
    with open(filename, 'wb') as f:
        f.flush()  # Ensure file is opened properly
        
        while written_bytes < data_size:
            # Create chunk of random data
            chunk = os.urandom(chunk_size)
            
            # Write chunk
            f.write(chunk)
            written_bytes += len(chunk)
            chunk_count += 1
            
            # Force write to disk immediately
            f.flush()
            os.fsync(f.fileno())
            
            # Calculate progress
            elapsed_time = time.time() - start_time
            if elapsed_time > 0:
                current_speed = written_bytes / (1024 * 1024) / elapsed_time  # MB/s
                print(f"Writing: {written_bytes / (1024**3):.2f} GB ({chunk_count} chunks), "
                      f"Speed: {current_speed:.2f} MB/s", end='\r')
            
            # Sleep to maintain 100MB/s rate (10ms per chunk)
            time.sleep(target_time_per_chunk)
    
    print(f"\nWritten {os.path.getsize(filename) / (1024**3):.2f} GB to disk")
    
    # Force OS to flush all buffers
    print("Forcing OS to flush all buffers...")
    os.sync()
    
    # Wait 10 seconds
    print("Waiting 10 seconds...")
    time.sleep(10)
    
    # Read data back with forced cache bypass
    print("Reading 1GB data back (forced cache bypass)...")
    start_time = time.time()
    read_bytes = 0
    read_chunk_count = 0
    
    # Method 1: Read with forced cache invalidation using posix_fadvise
    try:
        with open(filename, 'rb') as f:
            while read_bytes < data_size:
                # Read chunk
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                    
                read_bytes += len(chunk)
                read_chunk_count += 1
                
                # Force OS to read from disk, not cache
                # This is the closest we can get to bypassing cache
                try:
                    os.posix_fadvise(f.fileno(), 0, 0, os.POSIX_FADV_DONTNEED)
                except:
                    # POSIX_FADV_DONTNEED not available on all systems
                    pass
                
                # Calculate progress
                elapsed_time = time.time() - start_time
                if elapsed_time > 0:
                    current_speed = read_bytes / (1024 * 1024) / elapsed_time  # MB/s
                    print(f"Reading: {read_bytes / (1024**3):.2f} GB ({read_chunk_count} chunks), "
                          f"Speed: {current_speed:.2f} MB/s", end='\r')
                
                # Sleep to maintain 100MB/s rate
                time.sleep(target_time_per_chunk)
                
    except Exception as e:
        print(f"Error during read: {e}")
        # Fallback: simple read without cache manipulation
        with open(filename, 'rb') as f:
            while read_bytes < data_size:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                read_bytes += len(chunk)
                read_chunk_count += 1
                
                # Calculate progress
                elapsed_time = time.time() - start_time
                if elapsed_time > 0:
                    current_speed = read_bytes / (1024 * 1024) / elapsed_time  # MB/s
                    print(f"Reading: {read_bytes / (1024**3):.2f} GB ({read_chunk_count} chunks), "
                          f"Speed: {current_speed:.2f} MB/s", end='\r')
                
                # Sleep to maintain 100MB/s rate
                time.sleep(target_time_per_chunk)
    
    print(f"\nRead {read_bytes / (1024**3):.2f} GB from disk")
    
    # Force OS to flush after reading
    print("Forcing OS to flush after reading...")
    os.sync()
    
    # Clean up
    os.remove(filename)
    print("1GB forced write/read operation completed")

# Execute the forced write/read operation
write_and_read_1gb_forced()

Starting 1GB write operation...
Writing: 1.00 GB (1024 chunks), Speed: 55.44 MB/s
Written 1.00 GB to disk
Forcing OS to flush all buffers...
Waiting 10 seconds...
Reading 1GB data back (forced cache bypass)...
Reading: 1.00 GB (1024 chunks), Speed: 80.61 MB/s
Read 1.00 GB from disk
Forcing OS to flush after reading...
1GB forced write/read operation completed
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 41.21s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  2.77     0.00     12.49    -       
Memory (GB)               2.64     2.64     2.64     23.13   


In [14]:
%perfmonitor_plot

In [15]:
# Cell 6: CUDA Memory Operations (if CUDA available)
try:
    import torch
    
    def cuda_memory_operations():
        """Fill 2GB of CUDA memory, wait 10sec, then empty it"""
        if not torch.cuda.is_available():
            print("CUDA not available, skipping CUDA memory operations")
            return
            
        print("CUDA available, performing memory operations...")
        device = torch.device('cuda')
        
        # Calculate number of elements for 2GB (assuming float32 = 4 bytes each)
        gb_size = 2 * 1024 * 1024 * 1024  # 2 GB in bytes
        num_elements = int(gb_size / 4)  # 4 bytes per float32 element
        
        print(f"Allocating 2GB of CUDA memory ({num_elements} elements)")
        
        # Allocate memory on GPU
        tensor = torch.randn(num_elements, dtype=torch.float32, device=device)
        print(f"Allocated tensor size: {tensor.numel() * 4 / (1024**3):.2f} GB")
        
        # Wait 10 seconds
        print("Waiting 10 seconds...")
        time.sleep(10)
        
        # Free memory
        del tensor
        torch.cuda.empty_cache()
        print("CUDA memory freed")
        
        print("CUDA memory operations completed")
    
    # Execute CUDA memory operations
    cuda_memory_operations()
    
except ImportError:
    print("PyTorch not installed, skipping CUDA operations")
except Exception as e:
    print(f"CUDA operation error: {e}")

PyTorch not installed, skipping CUDA operations
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 41.21s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  2.77     0.00     12.49    -       
Memory (GB)               2.64     2.64     2.64     23.13   


In [16]:
%perfmonitor_plot

In [17]:
# Cell 7: CUDA Heavy Computation (if CUDA available)
try:
    import torch
    
    def cuda_heavy_computation():
        """Perform heavy computation on CUDA device for 20 seconds"""
        if not torch.cuda.is_available():
            print("CUDA not available, skipping CUDA heavy computation")
            return
            
        print("CUDA available, performing heavy computation...")
        device = torch.device('cuda')
        
        # Create large tensors for computation
        size = 10000  # Size of square matrices
        iterations = 0
        
        print(f"Starting heavy computation on CUDA for 20 seconds...")
        start_time = time.time()
        
        # Continue computation until 20 seconds have passed
        while time.time() - start_time < 20:
            # Create random matrices
            a = torch.randn(size, size, dtype=torch.float32, device=device)
            b = torch.randn(size, size, dtype=torch.float32, device=device)
            
            # Matrix multiplication (heavy operation)
            c = torch.matmul(a, b)
            
            # Additional computation to increase workload
            d = torch.sin(c) + torch.cos(c)
            
            # More operations to keep GPU busy
            e = torch.exp(d) * torch.log(d + 1e-8)
            
            # Increase iteration count
            iterations += 1
            
            # Clear cache periodically to prevent memory issues
            if iterations % 10 == 0:
                torch.cuda.empty_cache()
        
        print(f"Heavy computation completed in 20 seconds with {iterations} iterations")
        print("CUDA heavy computation completed")
    
    # Execute CUDA heavy computation
    cuda_heavy_computation()
    
except ImportError:
    print("PyTorch not installed, skipping CUDA heavy computation")
except Exception as e:
    print(f"CUDA heavy computation error: {e}")

PyTorch not installed, skipping CUDA heavy computation
----------------------------------------
JUmPER Performance Report
----------------------------------------
Duration: 41.21s (1 cell)
----------------------------------------
Metric                    AVG      MIN      MAX      TOTAL   
-----------------------------------------------------------------
CPU Util (Across 8 CPUs)  2.77     0.00     12.49    -       
Memory (GB)               2.64     2.64     2.64     23.13   


In [18]:
%perfmonitor_plot